Questo notebook è fortemente ispirato all'articolo *A Kalman Filtering Tutorial for Undergraduate Students*.

### Riferimenti bibliografici:

* Rhudy, M., B.; Salguero, R., A. & Holappa, K. (2017), [A Kalman Filtering Tutorial for Undergraduate Students](http://aircconline.com/ijcses/V8N1/8117ijcses01.pdf).

# Modello lineare dinamico e filtro di Kalman con NumPy

## Indice

1. [Modello lineare dinamico](#modello_lineare_dinamico)<br />
2. [Filtro di Kalman](#kalman)<br />
3. [Esempio](#esempio)<br />
    3.1 [Oggetto in caduta libera senza resistenza aerea](#oggetto_caduta)<br />
    3.2 [Termini del problema](#termini)<br />
    3.3 [Generazione delle serie degli stati e degli output](#generazione)<br />
    3.4 [Simulazione di misurazioni tramite l'aggiunta di rumore a y](#misurazioni)<br />
    3.5 [Filtraggio di Kalman](#filtraggio)<br />
    3.6 [Grafico degli stati veri e stimati](#grafico)<br />

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import inspect

%load_ext autoreload
%autoreload 2

# 1. Modello lineare dinamico <a id=modello_lineare_dinamico> </a>

$$
\begin{align}
    \textbf{x}_k &= \textbf{F}_{k-1}\textbf{x}_{k-1} + \textbf{G}_{k-1}\textbf{u}_{k-1} + \textbf{w}_k \tag{1} \\
    \textbf{y}_k &= \textbf{H}_k\textbf{x}_k  + \textbf{v}_k \tag{2}
\end{align}
$$

### Dimensioni delle variabili del sistema a tempo discreto

| Variabile  | Descrizione                          | Dimensione     |
|------------|--------------------------------------|----------------|
|$\textbf{x}$| Vettore di stato                     |$n_x \times 1$  |
|$\textbf{y}$| Vettore di output                    |$n_y \times 1$  |
|$\textbf{u}$| Vettore di input                     |$n_u \times 1$  |
|$\textbf{w}$| Vettore del rumore del processo      |$n_x \times 1$  |
|$\textbf{v}$| Vettore del rumore di misurazione    |$n_y \times 1$  |
|$\textbf{F}$| Matrice del sistema - stato          |$n_x \times n_x$|
|$\textbf{G}$| Matrice del sistema - input          |$n_x \times n_u$|
|$\textbf{H}$| Matrice delle osservazioni           |$n_y \times n_x$|
|$\textbf{P}$| Matrice di covarianza di $\textbf{x}$|$n_x \times n_x$|
|$\textbf{Q}$| Matrice di covarianza di $\textbf{w}$|$n_x \times n_x$|
|$\textbf{R}$| Matrice di covarianza di $\textbf{v}$|$n_y \times n_y$|
|$\textbf{z}$| Misurazione di $\textbf{y}$          |$n_y \times 1$  |

# 2. Filtro di Kalman <a id=kalman> </a>

**Algoritmo** filtro di Kalman

$\hat{\textbf{x}}_0$: stima iniziale del vettore di stato. <br />
$\hat{\textbf{P}}_0$: stima iniziale della matrice di covarianza dell'errore di stato.

1. $\hat{\textbf{x}}_{k|k-1} = \textbf{F}_{k-1}\hat{\textbf{x}}_{k-1} + \textbf{G}_{k-1}\textbf{u}_{k-1},\quad \textit{vettore di stato previsto}$;
2. $\hat{\textbf{P}}_{k|k-1} = \textbf{F}_{k-1}\hat{\textbf{P}}_{k-1}\textbf{F}^\top_{k-1} + \textbf{Q}_{k-1},\quad \textit{matrice di covarianza dell'errore di stato prevista}$;
3. $\textbf{K}_k = \hat{\textbf{P}}_{k|k-1}\textbf{H}^\top_k(\textbf{H}_k\hat{\textbf{P}}_{k|k-1}\textbf{H}^\top_k + \textbf{R}_k)^{-1},\quad \textit{matrice del guadagno di Kalman}$;
4. $\hat{\textbf{x}}_{k} = \hat{\textbf{x}}_{k|k-1} + \textbf{K}_{k}(\textbf{z}_k - \textbf{H}_k\hat{\textbf{x}}_{k|k-1}),\quad \textit{correzione del vettore di stato previsto}$;
5. $\hat{\textbf{P}}_{k} = (\textbf{I} - \textbf{K}_k\textbf{H}_k)\hat{\textbf{P}}_{k|k-1},\, \textbf{I}\, \text{matrice identità},\quad \textit{correzione della matrice di covarianza dell'errore di stato prevista}$.

# 3. Esempio <a id=esempio> </a>

## 3.1 Oggetto in caduta libera senza resistenza aerea <a id=oggetto_caduta> </a>

**Obiettivo**: determinare l'altezza $h$ di un oggetto in caduta libera senza resistenza aerea, in base all'informazione incerta riguardo la sua altezza iniziale e a delle misure della sua posizione a tempi successivi fornite da un telemetro laser.

Sia $g$ l'accelerazione dovuta alla gravità ($g = 9.80665\, m/s^2$), usando la cinematica delle particelle abbiamo che

$\ddot{h}(t) = -g,\quad \textit{accelerazione}\, a(t) = a \label{1}\tag{1}$

Dall'equazione che relaziona velocità e accelerazione $v(t) = \int_{t_0}^t a(t)\, dt$, quando $a(t)$ è costante come nel nostro caso, abbiamo che $v(t) = v(t_0) + a\Delta t$ con $\Delta t = t - t_0$ (e quindi $t_0 = t - \Delta t$), otteniamo

$\dot{h}(t) =\dot{h}(t_0) - g\Delta t,\quad \textit{velocità}\, v(t) \label{2}\tag{2}$

Integrando di nuovo otteniamo l'equazione del *moto uniformemente accelerato* $s(t) = \int_{t_0}^t a\Delta t + v(t_0)\, dt = s(t_0) + v(t_0)\Delta t + \frac{1}{2}a\Delta t^2$. Nel nostro caso

$h(t) = h(t_0) + \dot{h}(t_0)\Delta t -\frac{1}{2}g(\Delta t)^2,\quad \textit{posizione}\, s(t) \label{3}\tag{3}$

Passando ad un indice di tempo discreto $t = k \Delta t$ e ponendo $h_k = h(k \Delta t)$, possiamo riscrivere $\eqref{2}$ e $\eqref{3}$ come

$\dot{h}_k =\dot{h}_{k-1} - g\Delta t,\quad \textit{velocità}\, v_k \label{4}\tag{4}$

$h_k = h_{k-1} + \dot{h}_{k-1} -\frac{1}{2}g(\Delta t)^2,\quad \textit{posizione}\, s_k \label{5}\tag{5}$

Siamo interessato alla stima della posizione dell'oggetto, sappiamo quindi che la posizione deve essere inclusa nel vettore di stato. Tuttavia, dato che anche la velocità appare nell'equazione della posizione, è necessario ottenere anche questa informazione. Un modo per averla è includere la velocità nel vettore di stato. Definiamo quindi il vettore di stato del filtro di Kalman come

$\textbf{x}_k = \begin{pmatrix} h_k \\ \dot{h}_k \end{pmatrix}\label{6}\tag{6}$

che risulta nella seguente espressione

$\textbf{x}_k = \begin{pmatrix} h_{k-1} + \dot{h}_{k-1} -\frac{1}{2}g(\Delta t)^2 \\ \dot{h}_{k-1} - g\Delta t \end{pmatrix}\label{7}\tag{7}$

Con questa definizione, possiamo riscrivere queste equazioni in termini del vettore di stato, $\textbf{x}$, come in

$\textbf{x}_k = \begin{pmatrix} 1 & \Delta t \\ 0 & 1 \end{pmatrix}\textbf{x}_{k-1} + \begin{pmatrix} -\frac{1}{2}(\Delta t)^2 \\ -\Delta t \end{pmatrix}g \label{8}\tag{8}$

Abbiamo quindi il problema nella forma necessaria per applicare il filtro di Kalman

$\textbf{x}_k = \textbf{F}_{k-1}\textbf{x}_{k-1} + \textbf{G}_{k-1}\textbf{u}_{k-1} + \textbf{w}_k \label{9}\tag{9}$

$\textbf{F}_{k-1} = \begin{pmatrix} 1 & \Delta t \\ 0 & 1 \end{pmatrix},\, \textbf{G}_{k-1} = \begin{pmatrix} -\frac{1}{2}(\Delta t)^2 \\ -\Delta t \end{pmatrix},\, \textbf{u}_{k-1} = g, \textbf{w}_k = \begin{pmatrix} 0 \\ 0 \end{pmatrix} \label{10}\tag{10}$

Ponendo $\textbf{w}_k$ uguale a zero stiamo assumendo che non ci sia errore nell'equazione stessa. Per questo problema, questa è un'assunzione, dato che ci potrebbe essere un disturbo dovuto alla resistenza dell'aria o ad altre fonti. Se il caso generale con rumore gaussiano è $\textbf{w}_k \sim \mathcal{N}(0, \textbf{Q}_k)$, stiamo quindi ponendo

$\textbf{Q}_k = \begin{pmatrix} 0 & 0 \\ 0 & 0 \end{pmatrix} \label{11}\tag{11}$

Consideriamo ora la parte relativa alle misurazioni del sistema.

Immaginiamo uno scenario dove la posizione dell'oggetto può essere misurata con un telemetro laser con una deviazione standard dell'errore $\textbf{v}$ pari a 2 $m$. La matrice di covarianza di $\textbf{v}$, $\textbf{R}$, è quindi pari a $(2\,m)^2 = 4\,m^2$. Notare che sia $\textbf{v}$ che $\textbf{R}$ sono in questo caso degli scalari essendoci solo un termine nel vettore di output $\textbf{y}$ dato dalla relazione

$\textbf{y}_k = h_k  + \textbf{v}_k \label{12}\tag{12}$

che possiamo riscrivere come

$\textbf{y}_k = \begin{pmatrix} 1 & 0 \end{pmatrix}\textbf{x}_k + \textbf{v}_k \label{13}\tag{13}$

Rispetto all'equazione generica

$\textbf{y}_k = \textbf{H}_k\textbf{x}_k  + \textbf{v}_k \label{14}\tag{14}$

nel nostro caso abbiamo che

$\textbf{H}_k = \begin{pmatrix} 1 & 0 \end{pmatrix} \label{15}\tag{15}$

Sia la vera altezza inziale $h_0 = 100\, m$ e sia la nostra stima di essa $\hat{h}_0 = \hat{\textbf{x}}_0[0]  = 105\, m$. Dato che abbiamo stimato a occhio la posizione inziale, cosideriamo una varianza relativamente alta dell'errore commesso, $\hat{\textbf{P}}_0[0, 0] = 10\, m^2$. Assumiamo che l'oggetto parta da fermo $\hat{\dot{h}}_0 = \hat{\textbf{x}}_0[1] = 0\, m/s$. Quest'assunzione è piuttosto ragionevole, possiamo quindi considerare una piccola incertezza, $\hat{\textbf{P}}_0[1, 1] = 0.01\, m^2/s^2$. Infine immaginiamo di prendere $N = 1000$ misurazioni e che $\Delta t$ sia pari a $0.001\, s$.

## 3.2 Termini del problema <a id=termini> </a>

In [ ]:
u = np.array([[9.80665]]) # vettore (scalare in questo caso) di input
dt = 0.001 # tempo di campionamento (s)
F = np.array( # matrice del sistema - stato
    [[1, dt],
     [0, 1]]
)
G = np.array( # matrice (vettore in questo caso) del sistema - input
    [[-1 / 2 * dt ** 2],
     [-dt]]
)
H = np.array([[1, 0]]) # matrice (vettore in questo caso) delle osservazioni
Q = np.zeros((2, 2)) # matrice di covarianza dell'errore del processo w
R = np.array([[4]]) # matrice di covarianza (scalare in questo caso) dell'errore di output v
x0 = np.array( # vettore del vero stato inziale
    [[100],
     [0]]
)
x0_hat = np.array( # vettore di stato inziale ipotizzato
    [[105],
     [0]]
)
P0_hat = np.array([ # matrice di covarianza inziale del vettore di stato ipotizzata
    [10, 0],
    [0, 0.01]]
)
N = 1000 # numero di passi temporali
t = dt * np.arange(0, N + 1) # vettore del tempo (s)

### Esercizio

1. Completare il metodo `eq1` della classe `ModelloLineareDinamico` in `msbd/modello_lineare/modello_lineare_dinamico.py`;
2. Completare il metodo `eq2` della classe `ModelloLineareDinamico` in `msbd/modello_lineare/modello_lineare_dinamico.py`;
2. Scrivere dei test per verificare che i metodi implementati siano corretti.

In [ ]:
from msbd.modello_lineare import ModelloLineareDinamico

print(inspect.getsource(ModelloLineareDinamico))

## 3.3 Generazione delle serie degli stati e degli output <a id=generazione> </a>

In [ ]:
x = x0
y = np.array([[]])

for k in range(1, N + 1):
    x_k = ModelloLineareDinamico.eq1(F, x[:, [k - 1]], G, u)
    x = np.hstack([x, x_k])
    
    y_k = ModelloLineareDinamico.eq2(H, x[:, [k]])
    y = np.hstack([y, y_k])

## 3.4 Simulazione di misurazioni tramite l'aggiunta di rumore a y <a id=misurazioni> </a>

In [ ]:
np.random.seed(42)

v = np.random.multivariate_normal(np.zeros(1), cov=R, size=N).reshape(y.shape) # nota: in questo caso bastava np.random.normal

z = y + v

### Esercizio

1. Completare il metodo `partial_fit` della classe `ModelloLineareDinamico` in `msbd/modello_lineare/modello_lineare_dinamico.py`;
2. Scrivere dei test per verificare che il metodo implementato sia corretto.

## 3.5 Filtraggio di Kalman <a id=filtraggio> </a>

In [ ]:
fk = ModelloLineareDinamico(x0_hat, P0_hat)

for k in range(0, N):
    fk.partial_fit(F, G, u, H, Q, R, z[:, [k]])
    
h_hat = np.array(fk.x_hat_)[:, 0, 0] # array delle altezze
v_hat = np.array(fk.x_hat_)[:, 1, 0] # array delle velocità

## 3.6 Grafico degli stati veri e stimati <a id=grafico> </a>

In [ ]:
plt.figure(figsize=(8, 6))

plt.suptitle("Esempio di filtraggio di Kalman, stati veri e stimati")

plt.subplot(211)
plt.plot(t[1:], z[0, :], color="lime", label="Misurata")
plt.plot(t, h_hat, ls='--', color="blue", lw=3, label="Stimata")
plt.plot(t, x[0, :], ls=':', color="red", lw=2, label="Vera")
plt.ylabel("$x_0 = altezza\, (m)$")
plt.grid(ls='--')
plt.legend()

plt.subplot(212)
plt.plot(t, v_hat, ls='--', color="blue", lw=3, label="Stimata")
plt.plot(t, x[1, :], ls=':', color="red", lw=2, label="Vera")
plt.xlabel("secondi ($s$)")
plt.ylabel("$x_1 = velocità \, (m/s)$")
plt.grid(ls='--')
plt.legend()

plt.show()